In [13]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from catboost import CatBoostRegressor
import joblib

## experimental train process
i am going to train few regression models evaluate them and pick the best one for the mission

In [14]:
process_df = pd.read_csv("C:\\Users\\moran\\personal_projects\\vfm_car_finder\\VFM_car_finder\\processed_data.csv")
print(process_df.shape)
process_df.head()

(40556, 24)


,listing_id,upload_date,scrape_date,price,title,owner_count,mileage,color,ownership,test_date,...,seats,horsepower,engine_volume,fuel_consumption,drive_type,url,months_on_road,on_road_year,on_road_month,months_to_test
0,46544667,2025-05-21,25/05/2025,289999.0,ג'יפ גרנד צ'ירוקי,2,70000,כסוף,פרטית,2026-01-05,...,5,295,3604.0,10.00,4X4,https://www.yad2.co.il/vehicles/item/xh2tdvmd?...,36,2022,5,8.0
1,72039716,2025-01-09,25/05/2025,62900.0,מאזדה 2,2,88000,אפור מטאלי,פרטית,2026-01-01,...,5,116,1496.0,20.83,4X2,https://www.yad2.co.il/vehicles/item/iizgzpne?...,60,2020,1,12.0
2,65607492,2025-05-08,25/05/2025,210000.0,ב מ וו סדרה 7,3,110000,שחור,פרטית,NaN,...,5,326,1998.0,40.00,4X4,https://www.yad2.co.il/vehicles/item/vh8136jp?...,83,2018,6,NaN
3,74300037,2025-05-24,25/05/2025,46600.0,קיה ספורטז',2,182000,אפור,פרטית,2026-01-01,...,5,133,1591.0,14.71,4X2,https://www.yad2.co.il/vehicles/item/pxog95kb?...,112,2016,1,8.0
4,82186481,2025-05-16,25/05/2025,80000.0,מרצדס-בנץ CLA,5,170000,לבן,פרטית,2026-01-03,...,5,211,1991.0,17.24,4X2,https://www.yad2.co.il/vehicles/item/nxv8x4jj?...,110,2016,3,8.0


In [15]:
# Columns to exclude
exclude_cols = ['listing_id', 'url', 'scrape_date', 'upload_date', 'test_date', 'on_road_date']

# Define features and target
features = [col for col in process_df.columns if col not in exclude_cols + ['price']]
x = process_df[features]
y = process_df['price']
    
# Split the data
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,         # 20% for testing
    random_state=42,       # Ensures reproducibility
    shuffle=True           # Shuffle data before splitting
)

categorical_features = ['title', 'color', 'transmission', 'fuel_type', 'body_type', 'ownership']
for col in categorical_features:
    x_train[col] = x_train[col].astype('category')
    x_test[col] = x_test[col].astype('category')


print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
x_train

(32444, 17)
(8112, 17)
(32444,)
(8112,)


,title,owner_count,mileage,color,ownership,transmission,fuel_type,body_type,seats,horsepower,engine_volume,fuel_consumption,drive_type,months_on_road,on_road_year,on_road_month,months_to_test
17518,מאזדה 3,3,235000,זהב,פרטית,אוטומטי,בנזין,סדאן,5,105,1598.0,13.16,4X2,204,2008,1,0.0
30360,טויוטה פריוס,2,330000,כסוף מטאלי,פרטית,אוטומטי,היברידי בנזין,האצ'בק,5,99,1798.0,25.00,4X2,182,2010,3,8.0
2004,רנו קנגו,1,150000,לבן שנהב,ליסינג,אוטומטי,בנזין,מיניוואן קומפקטי,5,115,1197.0,16.39,4X2,82,2018,3,12.0
963,טויוטה RAV4,2,80000,לבן פנינה,פרטית,אוטומטי,היברידי בנזין,פנאי-שטח,5,155,2494.0,20.00,4X2,76,2019,1,8.0
5685,מאזדה 3,1,187780,אפור עכבר,פרטית,אוטומטי,בנזין,סדאן,5,150,1999.0,12.66,4X2,191,2009,6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,מאזדה CX-30,1,46000,לבן שנהב,פרטית,אוטומטי,בנזין,פנאי-שטח,5,165,1998.0,18.18,4X2,49,2021,3,9.0
11284,מיני קופר,3,170000,קרם,פרטית,אוטומטי,בנזין,האצ'בק,4,231,1998.0,17.54,4X2,115,2015,8,0.0
38158,מרצדס-בנץ GLE,2,128000,שחור,פרטית,אוטומטי,פלאג-אין בנזין,פנאי-שטח,5,333,2996.0,27.03,4X4,90,2017,8,0.0
860,ג׳ילי Geometry C,1,30000,כסוף,ליסינג,אוטומטי,חשמלי,פנאי-שטח,5,204,NaN,NaN,4X2,15,2024,3,7.0


In [16]:
['owner_count', 'mileage', 'seats', 'horsepower', 'engine_volume', 'fuel_consumption' , 'on_road_year', 'on_road_month', 'months_to_test']
['title', 'color', 'transmission', 'fuel_type', 'body_type', 'ownership']

['title', 'color', 'transmission', 'fuel_type', 'body_type', 'ownership']

in the last part we saw a closer to linear relation of the price with sqrt(milage) so i will try 2 train data frame with sqrt and without

In [17]:
x_sqrt_train = x_train.copy()
x_sqrt_test = x_test.copy()

x_sqrt_train['sqrt_milage'] = np.sqrt(x_sqrt_train['mileage']) 
x_sqrt_train.drop(['mileage'], axis=1, inplace=True)

x_sqrt_test['sqrt_milage'] = np.sqrt(x_sqrt_test['mileage']) 
x_sqrt_test.drop(['mileage'], axis=1, inplace=True)

print(x_sqrt_train.shape)
print(x_sqrt_test.shape)

(32444, 17)
(8112, 17)


creat preprocess pipline to fill missing values

In [18]:
# STEP 1: Define your features
numeric_features = ['owner_count', 'mileage', 'seats', 'horsepower', 'engine_volume',
                    'fuel_consumption', 'on_road_year', 'on_road_month', 'months_to_test']
categorical_features = ['title', 'color', 'transmission', 'fuel_type', 'body_type', 'ownership']



numeric_features_sqrt = ['owner_count', 'sqrt_milage', 'seats', 'horsepower', 'engine_volume',
                         'fuel_consumption', 'on_road_year', 'on_road_month', 'months_to_test']

# STEP 2: Define your pipelines
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# STEP 3: ColumnTransformers for both variants
preprocessor_raw = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

preprocessor_sqrt = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features_sqrt),
    ('cat', categorical_pipeline, categorical_features)
])

# STEP 4: FIT the pipelines and TRANSFORM the raw data
# IMPORTANT: fit only on TRAINING data
preprocessor_raw.fit(x_train)
x_train_arr = preprocessor_raw.transform(x_train)
x_test_arr = preprocessor_raw.transform(x_test)

preprocessor_sqrt.fit(x_sqrt_train)
x_sqrt_train_arr = preprocessor_sqrt.transform(x_sqrt_train)
x_sqrt_test_arr = preprocessor_sqrt.transform(x_sqrt_test)

# STEP 5: Convert arrays back to DataFrames with column names
x_train = pd.DataFrame(x_train_arr, columns=numeric_features + categorical_features)
x_test = pd.DataFrame(x_test_arr, columns=numeric_features + categorical_features)
x_sqrt_train = pd.DataFrame(x_sqrt_train_arr, columns=numeric_features_sqrt + categorical_features)
x_sqrt_test = pd.DataFrame(x_sqrt_test_arr, columns=numeric_features_sqrt + categorical_features)

# STEP 6: Mark categorical columns as category dtype (for LightGBM, XGBoost, etc.)
for col in categorical_features:
    x_train[col] = x_train[col].astype('category')
    x_test[col] = x_test[col].astype('category')
    x_sqrt_train[col] = x_sqrt_train[col].astype('category')
    x_sqrt_test[col] = x_sqrt_test[col].astype('category')

for col in numeric_features:
    x_train[col] = x_train[col].astype('float')
    x_test[col] = x_test[col].astype('float')

for col in numeric_features_sqrt:
    x_sqrt_train[col] = x_sqrt_train[col].astype('float')
    x_sqrt_test[col] = x_sqrt_test[col].astype('float')    

# build a basline model 
I chose to use simple linear regression model only on the numeric features that will use as a baseline to test the more advanced models

In [19]:
x_train_numeric_only = x_train[numeric_features]
x_test_numeric_only = x_test[numeric_features]

numeric_only_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

numeric_only_pipeline.fit(x_train_numeric_only, y_train)
y_pred = numeric_only_pipeline.predict(x_test_numeric_only)

print("🔹 Numeric-Only Baseline (True Numeric Columns Only)")
print(f"MAE:  {mean_absolute_error(y_test, y_pred):.2f}")
print(f"RMSE: {mean_squared_error(y_test, y_pred):.2f}")
print(f"R²:   {r2_score(y_test, y_pred):.4f}")

🔹 Numeric-Only Baseline (True Numeric Columns Only)
MAE:  31056.18
RMSE: 3171735514.51
R²:   0.6182


### evaluate a few models that can handle categorical features on their default to pick one for fine tuning

In [20]:
def evaluate(model, x_test, y_test, label=""):
    # Predict
    y_pred = model.predict(x_test)
    
    # Evaluate
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"🔹 Results for: {label}")
    print(f"MAE:  {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²:   {r2:.4f}\n")
    
    return model  # Return trained model if needed


In [21]:
x_train

,owner_count,mileage,seats,horsepower,engine_volume,fuel_consumption,on_road_year,on_road_month,months_to_test,title,color,transmission,fuel_type,body_type,ownership
0,3.0,235000.0,5.0,105.0,1598.0,13.16,2008.0,1.0,0.0,מאזדה 3,זהב,אוטומטי,בנזין,סדאן,פרטית
1,2.0,330000.0,5.0,99.0,1798.0,25.00,2010.0,3.0,8.0,טויוטה פריוס,כסוף מטאלי,אוטומטי,היברידי בנזין,האצ'בק,פרטית
2,1.0,150000.0,5.0,115.0,1197.0,16.39,2018.0,3.0,12.0,רנו קנגו,לבן שנהב,אוטומטי,בנזין,מיניוואן קומפקטי,ליסינג
3,2.0,80000.0,5.0,155.0,2494.0,20.00,2019.0,1.0,8.0,טויוטה RAV4,לבן פנינה,אוטומטי,היברידי בנזין,פנאי-שטח,פרטית
4,1.0,187780.0,5.0,150.0,1999.0,12.66,2009.0,6.0,0.0,מאזדה 3,אפור עכבר,אוטומטי,בנזין,סדאן,פרטית
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32439,1.0,46000.0,5.0,165.0,1998.0,18.18,2021.0,3.0,9.0,מאזדה CX-30,לבן שנהב,אוטומטי,בנזין,פנאי-שטח,פרטית
32440,3.0,170000.0,4.0,231.0,1998.0,17.54,2015.0,8.0,0.0,מיני קופר,קרם,אוטומטי,בנזין,האצ'בק,פרטית
32441,2.0,128000.0,5.0,333.0,2996.0,27.03,2017.0,8.0,0.0,מרצדס-בנץ GLE,שחור,אוטומטי,פלאג-אין בנזין,פנאי-שטח,פרטית
32442,1.0,30000.0,5.0,204.0,1591.0,16.67,2024.0,3.0,7.0,ג׳ילי Geometry C,כסוף,אוטומטי,חשמלי,פנאי-שטח,ליסינג


In [22]:
print('x_train: ', x_train.dtypes)
print('x_test: ', x_test.dtypes)


x_train:  owner_count          float64
mileage              float64
seats                float64
horsepower           float64
engine_volume        float64
fuel_consumption     float64
on_road_year         float64
on_road_month        float64
months_to_test       float64
title               category
color               category
transmission        category
fuel_type           category
body_type           category
ownership           category
dtype: object
x_test:  owner_count          float64
mileage              float64
seats                float64
horsepower           float64
engine_volume        float64
fuel_consumption     float64
on_road_year         float64
on_road_month        float64
months_to_test       float64
title               category
color               category
transmission        category
fuel_type           category
body_type           category
ownership           category
dtype: object


In [23]:
models = {
    "LightGBM": LGBMRegressor(
        n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42,verbosity=-1
    ),
    "XGBoost": XGBRegressor(
        n_estimators=100, learning_rate=0.1, max_depth=3,
        random_state=42, enable_categorical=True
    ),
    "CatBoost": CatBoostRegressor(
        iterations=100, learning_rate=0.1, depth=6, verbose=0, random_state=42
    )
}

models["LightGBM"].fit(x_train, y_train)
models["XGBoost"].fit(x_train, y_train)

cat_features = x_train.select_dtypes(include="category").columns.tolist()
models["CatBoost"].fit(x_train, y_train, cat_features=cat_features)


for name, model in models.items():
    evaluate(model, x_test, y_test, label=f"{name} - Raw Mileage")
    # evaluate(model, x_sqrt_train, x_sqrt_test, y_train, y_test, label=f"{name} - Sqrt Mileage")

models["LightGBM"].fit(x_sqrt_train, y_train)
models["XGBoost"].fit(x_sqrt_train, y_train)

cat_features = x_train.select_dtypes(include="category").columns.tolist()
models["CatBoost"].fit(x_sqrt_train, y_train, cat_features=cat_features)

for name, model in models.items():
    evaluate(model, x_sqrt_test, y_test, label=f"{name} - Sqrt Mileage")    


🔹 Results for: LightGBM - Raw Mileage
MAE:  12998.50
RMSE: 28977.61
R²:   0.8989

🔹 Results for: XGBoost - Raw Mileage
MAE:  39302.35
RMSE: 73164.30
R²:   0.3557

🔹 Results for: CatBoost - Raw Mileage
MAE:  16540.86
RMSE: 34048.81
R²:   0.8605
🔹 Results for: LightGBM - Sqrt Mileage
MAE:  12998.50
RMSE: 28977.61
R²:   0.8989

🔹 Results for: XGBoost - Sqrt Mileage
MAE:  39302.35
RMSE: 73164.30
R²:   0.3557

🔹 Results for: CatBoost - Sqrt Mileage
MAE:  16540.86
RMSE: 34048.81
R²:   0.8605


catBoost and LightGBM are way better than the baseline model and the XGBoost.
also there is no significant difference in the sqrt features so i will keep going with the catBoost and LightGBM in the model_train_2 notebook